In [1]:
import re
import os
import sys
import h5py
import time
import numpy as np
import scipy as sp
from utils import *
import mindspore as ms
from scipy.sparse import csr_matrix
from scipy.optimize import minimize
from mindquantum.framework import *
from mindquantum.core.gates import *
from scipy.stats import unitary_group
from mindquantum.core.circuit import *
from IPython.display import display_svg
from mindquantum.core.operators import *
from mindquantum.algorithm.nisq import *
from mindspore.common.initializer import *
from mindquantum.simulator import Simulator
from mindquantum.algorithm.compiler import *
from numpy.linalg import det, svd, eigh, norm
from scipy.linalg import expm, sqrtm, block_diag

np.set_printoptions(linewidth=200)

In [7]:
# Test whether a qudit encoding is in the symmetric subspace
def is_symmetric(mat: np.ndarray) -> bool:
    b = True
    nq_bin = {}
    d = mat.shape[0]
    nq = int(np.log2(d))
    for i in range(d):
        num1 = bin(i).count('1')
        if num1 in nq_bin:
            nq_bin[num1].append(i)
        else:
            nq_bin[num1] = [i]
    for i in range(nq):
        ii = nq_bin[i]
        for j in range(nq):
            jj = nq_bin[j]
            if len(ii) != 1 or len(jj) != 1:
                a = mat[np.ix_(ii, jj)]
                b = b & np.allclose(a, a[0])
    return b


d = 3
# qudit = np.random.rand(d, d)
qudit = np.reshape(range(d**2), [d, d])
qubit = su2_encoding(qudit)
print(qudit)
print(qubit)
is_symmetric(qubit)

[[0 1 2]
 [3 4 5]
 [6 7 8]]
[[0.        +0.j 0.70710678+0.j 0.70710678+0.j 2.        +0.j]
 [2.12132034+0.j 2.        +0.j 2.        +0.j 3.53553391+0.j]
 [2.12132034+0.j 2.        +0.j 2.        +0.j 3.53553391+0.j]
 [6.        +0.j 4.94974747+0.j 4.94974747+0.j 8.        +0.j]]


True

In [2]:
# How to restore a reduce density matrix
np.random.seed(42)
a = np.random.rand(2) + 1j * np.random.rand(2)
b = np.random.rand(2) + 1j * np.random.rand(2)
a /= norm(a)
b /= norm(b)
psi_ab = np.kron(a, b)
rho_a = np.outer(a, a.conj())
rho_b = np.outer(b, b.conj())
rho_ab = np.kron(rho_a, rho_b)
print(rho_a)
print(rho_b)
print(rho_ab)
print(psi_ab.conj() @ rho_ab @ psi_ab)
fidelity(rho_ab, psi_ab)

[[0.34880034+0.j         0.40977999+0.24334959j]
 [0.40977999-0.24334959j 0.65119966+0.j        ]]
[[0.03454462+0.j         0.0930421 -0.15714469j]
 [0.0930421 +0.15714469j 0.96545538+0.j        ]]
[[ 1.20491735e-02+0.j          3.24531155e-02-0.05481212j  1.41556919e-02+0.00840642j  7.63678855e-02-0.04175299j]
 [ 3.24531155e-02+0.05481212j  3.36751167e-01+0.j         -1.14305675e-04+0.0870365j   3.95624296e-01+0.23494318j]
 [ 1.41556919e-02-0.00840642j -1.14305675e-04-0.0870365j   2.24954416e-02+0.j          6.05889828e-02-0.10233257j]
 [ 7.63678855e-02+0.04175299j  3.95624296e-01-0.23494318j  6.05889828e-02+0.10233257j  6.28704218e-01+0.j        ]]
(1.0000000000000007+0j)


1.0000000000000002